<center><h1> Political party comments Parsing </h1></center>

# 1. Information

Данный notebook парсит комментарии групп VK следующих политических партий:   
**Единая Россия**,    
**ЛПР**,   
**Справедливая Россия**,   
**КПРФ**,  
**ЛДПР**,  
**Навальный**,  
**Партия Роста**,  
**РотФронт**,  
**Союз Марксистов**,  
**Яблоко**,  
**Патриоты России**,  
**Гражданская Платформа**.  
За основу взят код с https://github.com/DmitrySerg/top-russian-music.

# 2. Parsing VK Data

## 2.1 Import packages

In [13]:
import pickle                    # 
import requests                  # 
import pandas as pd              # 
import numpy as np               # 
import matplotlib.pyplot as plt  # 
import time            
from tqdm import tqdm_notebook   

In [14]:
#https://oauth.vk.com/authorize?client_id=xxxxxxx&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

# 2.2 Groups IDs from VK APIs

In [15]:
# My VK page
my_user_id = 1801800
# VK API version used
version = '5.120' 
# Session token
with open('token') as f:
    token = f.read()

In [16]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Returns the result of a query by method
        
        method: string
            method to use
            
        parameters: string
            parameters of the used method
            
        token: string
            access token
        
        version: string
            API version
    """
    
    # making a link
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # request a link and translate into json
    response = requests.get(url).json()
    return response

vkDownload('users.get','user_ids=1801800')

{'response': [{'id': 1801800,
   'first_name': 'Алексей',
   'last_name': 'Гущин',
   'is_closed': False,
   'can_access_closed': True}]}

**Get parties IDs**

In [17]:
vk_group_name = ['er_ru', 'lpr_vk', 'spravo_ross', 'kprf', 'ldpr', 'teamnavalny',
                'partrost', 'rot_front', 'marxist_union', 'yabloko_ru',
                'rupatriots', 'civilplatform']
party_name = ['Единая Россия', 'ЛПР', 'Справедливая Россия', 'КПРФ', 'ЛДПР', 'Навальный', 
                                 'Партия Роста', 'РотФронт', 'Союз Марксистов', 'Яблоко',
                                 'Патриоты России', 'Гражданская Платформа']  
party_dict = {party_name[i]:vk_group_name[i] for i in range(len(vk_group_name))}

In [18]:
party_dict

{'Единая Россия': 'er_ru',
 'ЛПР': 'lpr_vk',
 'Справедливая Россия': 'spravo_ross',
 'КПРФ': 'kprf',
 'ЛДПР': 'ldpr',
 'Навальный': 'teamnavalny',
 'Партия Роста': 'partrost',
 'РотФронт': 'rot_front',
 'Союз Марксистов': 'marxist_union',
 'Яблоко': 'yabloko_ru',
 'Патриоты России': 'rupatriots',
 'Гражданская Платформа': 'civilplatform'}

In [19]:
group_ids=[]
for i in vk_group_name:
    group_ids.append(vkDownload('utils.resolveScreenName', 'screen_name={group_name}'.format(group_name = i)))
    time.sleep(0.3)
target_groups= {j:str(i['response']['object_id']) for i, j in zip(group_ids, party_name)}

In [13]:
target_groups

{'Единая Россия': '18483909',
 'ЛПР': '24070381',
 'Справедливая Россия': '113757409',
 'КПРФ': '26362316',
 'ЛДПР': '433349',
 'Навальный': '55284725',
 'Партия Роста': '85171267',
 'РотФронт': '502742',
 'Союз Марксистов': '65210882',
 'Гражданская Платформа': '63575',
 'Яблоко': '124134058',
 'Патриоты России': '52853267'}

# 2.3 Download Political Parties Group Comments

In [19]:
def getPosts(group_id):
    """
        Downloads the list of all posts in the 
        group by group and returns a list of their IDs.
    """
    
    post_ids = [ ] 
    
    # Number of posts 
    n = vkDownload('wall.get','owner_id=-{}'.format(group_id))['response']['count']
    print("В группе {} должно быть {} постов".format(group_id,n))
    
    for i in tqdm_notebook(range(0, n + 100, 100)):
        time.sleep(0.3)
        
        wall = vkDownload('wall.get','owner_id=-{}&count=100&offset='.format(group_id)+str(i))['response']['items'] 
        post_ids.extend([item['id'] for item in wall])
        
    return post_ids


def prepareComments(group_id, posts):
    """
        Download 25 requests at a time, this function prepares requests
        for subsequent use, understanding how many comments under which post,
        throwing out posts with zero comments and making triplets (group, post, offset)
    """

    triple_comments = [ ]
    # make batches of 25 from all posts to make it faster
    posts_batch = makeBatch(posts)

    for batch in tqdm_notebook(posts_batch):
        time.sleep(0.4)

        # preparing a request for comments from the batch, which will find out the number of comments under the post
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        # for each post from batch use the wall.getComments method
        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(group_id) + '","post_id":"' + str(bt) + '"}),'

        # make a request for the current 25 comments
        requests.get(begin + middle[:-1] + end)

        # how many comments are under the post 
        s = requests.get(begin + middle[:-1] + end)
        
        # based on the information obtained, we prepare requests for the future
        try:
            for post,item in zip(batch, s.json()['response']):
                if item is False:
                    print('False в посте {} группы {}'.format(post, group_id))
                    continue

                n = item['count']

                # post with zero comments ignore
                if n == 0:
                    continue 
                else:
                    # for every hundreds of comments from a post with comments there will be a request for extraction
                    for i in range(0, n, 100):
                        triple_comments.append((group_id, post, i))
        except:
            print("broken batch")
            continue
    return triple_comments


def getComments(triple_comments):
    """
        Divides vector elements of the form (group_id, post_id, offset),
        for batches of 15 pieces and downloads comments
    """
    
    comments = [ ] 

    precom_batch = makeBatch(triple_comments, 15)

    for batch in tqdm_notebook(precom_batch):
        time.sleep(0.4)
 
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(bt[0]) + '","post_id":"' + str(bt[1]) + '",\
            "need_likes":"1","preview_length":"0","count":"100","offset":"'+ str(bt[2])+'"}),'

        s = requests.get(begin + middle[:-1] + end)

        # unpacking
        cur_com = [ ]
        try:
            for item, post in zip(s.json()['response'], batch):
                for jtem in item['items']:
                    jtem['likes'] = jtem['likes']['count']
                    jtem['post_id'] = post[1]
                    jtem['group_id'] = post[0]
                    cur_com.append(jtem)
        except:
            pass
        comments.extend(cur_com) 
        
    return comments

In [42]:
group_names = {target_groups[k]:v for k,v in zip(target_groups, vk_group_name)}

In [45]:
unique_comments = { }

for name in target_groups:
    try:
        # download posts 
        group_id = target_groups[name]
        posts = getPosts(group_id)

        print('Из группы {} имени {} скачалось {} постов'.format(target_groups[name], name, len(posts)))

        # we prepare batches from requests for comments
        triple_comments = prepareComments(group_id, posts)

        print('Число запросов: {}'.format(len(triple_comments)))

        # download comment
        comments = getComments(triple_comments)

        setlen = len(set([str(comment['post_id']) + '_' + str(comment['id']) for comment in comments]))
        print('Число комментов: {}, из них уникальных: {}'.format(len(comments), setlen ))

        # look at the last ten comments 
        for comment in comments[-10:]:
            print(comment['post_id'],comment['id'])
            print(comment['text'])
            print('--------------------')

        # write in the dictionary
        unique_comments[name] = comments

        # Save
        with open('../data/{}_group_comments_{}'.format('politics', group_names[group_id]), 'wb') as f:
            pickle.dump(comments, f)

        print('====================================================================')
    except KeyboardInterrupt:
        break

В группе 18483909 должно быть 14296 постов



Из группы 18483909 имени Единая Россия скачалось 14296 постов



Число запросов: 8696



Число комментов: 122287, из них уникальных: 122287
435 323912
Спамьте под всеми постами
--------------------
435 323959
Спамьте под всеми постами
--------------------
435 324045
#Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт

████─███─█──█─█───█
--------------------
435 324274
#Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт #Бунт 

...


Число запросов: 213



Число комментов: 573, из них уникальных: 573
244 267
А ещё, Валентин, Багиров - не простая для восприятия фигура, конечно, но он как раз по социальным точкам накала проходится очень активно и бесстрашно. Он жертвует собственной репутацией, чтобы гипертрофировать важные проблемы. Ну, и не забывайте про ситуацию в Кишинёве и про то, что Багиров - доверенное лицо Владимира Путина. Не стоит искать линейные ответы, там где всё гораздо глубже)
--------------------
244 303
[club124134058|Патриоты России], так в этом и ответ- доверенное лицо такого человека патриотом быть не может
--------------------
226 227
Каким образом строительный олигарх Санкт-Петербурга, член партии "Единая Россия" Федор Туркин баллотируется от партии "ПАТРИОТЫ РОССИИ" в гос. Думу? 
В "Патриоты России" место кандидата дешевле чем в "Единая Россия" ?
--------------------
226 229
[id151382173|Фёдор], Ранее упомянутая вами принадлежность к другой партии еще никому не мешает быть настоящим патриотом.
Кстати, не поделитесь 


Из группы 52853267 имени Патриоты России скачалось 744 постов



Число запросов: 234



Число комментов: 1606, из них уникальных: 1606
107 110
😃 http://promodj.com/Almina.Star/blog
--------------------
78 80
Вы слышали? Всероссийский скандал! Бесчестное поведение московского уполномоченного по защите прав предпринимателей Михаила Вышегородцева уже вылезло на федеральный уровень! Вместо того, чтобы защищать предпринимателей, он их обманывает и кидает на деньги! Так что компании Никколо-М пришлось жаловаться аж Уполномоченному при Президенте РФ (они делали пиар-компанию Вышегородцеву, а тот зажал деньги). Получается, Вышегородцев дискредитирует весь институт защиты прав! Теперь рассматривается вопрос о прекращении деятельности всех уполномоченных по защите прав предпринимателей. И все из-за бесчестности и утраты доверия… Вышегородцев подставил всех!
--------------------
48 50
Общественная палата Москвы предлагает взять под полный контроль голосование вне участков. Остаётся уповать на честность и порядочность самих наблюдателей из ОП. А ещё лучше отправить за ними следить п